Segmenting and Clustering Neighborhoods in Toronto - Assignment

In [27]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -y lxml
import lxml.html as lh

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be UPDATED:

  ca-certificates     pkgs/main::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2020.4.5.1-hecc5488_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi              pkgs/main::certifi-2020.4.5.1-py36_0 --> conda-forge::certifi-2020.4.5.1-py36h9f0ad1d_0
  openssl              pkgs/main::openssl-1.1.1g-h7b6447c_0 --> conda-forge::openssl-1.1.1g-h516909a_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::

In [28]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# get page contents
page = requests.get(url)

# Store contents of the webpage
doc = lh.fromstring(page.content)

# row elements in html are stored between <tr>..</tr>
row_elements = doc.xpath('//tr')


In [29]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list name
for t in row_elements[0]:
    i+=1
    name=t.text_content()
    #print(name)
    col.append((name.rstrip(),[]))

In [30]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(row_elements)):
    #T is our j'th row
    T=row_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [31]:
# Organize the data into a pandas data frame
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df = df.replace('\n','', regex=True)

# drop cells with a borough that is Not assigned
df = df[df.Borough != "Not assigned"].reset_index(drop=True)

# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

# remove last text row - not relevant
df.drop(df.tail(1).index,inplace=True)

# assign completed frame to toronto_frame
toronto_frame=df
toronto_frame.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [32]:
# print the number of rows of your dataframe
toronto_frame.shape

(103, 3)

In [33]:
# get coordinates and assign to a dataframe
geo_coordinates = pd.read_csv("http://cocl.us/Geospatial_data")
geo_coordinates.rename(columns={"Postal Code": "Postal Code"}, inplace=True)
geo_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
#add coordinates to neighbourhood frame comparing the postal code to ensure accuracy

toronto_frame_postal = pd.merge(toronto_frame, geo_coordinates, how='outer', on=['Postal Code'])
toronto_frame_postal.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
